## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-01-20-53-27 +0000,nypost,Kash Patel severs FBI’s ties with James Comey-...,https://nypost.com/2025/10/01/us-news/kash-pat...
1,2025-10-01-20-50-28 +0000,nypost,NYPD cop snuck into Ryder Cup in full tactical...,https://nypost.com/2025/10/01/us-news/nypd-cop...
2,2025-10-01-20-48-26 +0000,startribune,Hundreds of Minnesota homes neglected by hedge...,https://www.startribune.com/hedge-fund-landlor...
3,2025-10-01-20-48-01 +0000,nypost,Senate Dems block bid to end government shutdo...,https://nypost.com/2025/10/01/us-news/senate-d...
4,2025-10-01-20-46-57 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/01/us/gov...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,government,42
45,shutdown,41
27,trump,40
180,will,20
126,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
222,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...,178
65,2025-10-01-18-55-00 +0000,wsj,Many GOP senators seeking an end to the govern...,https://www.wsj.com/politics/policy/white-hous...,169
323,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,154
28,2025-10-01-20-13-16 +0000,wapo,Trump administration seeks to exert control ov...,https://www.washingtonpost.com/business/2025/1...,133
184,2025-10-01-13-50-00 +0000,wsj,The White House and lawmakers failed to reach ...,https://www.wsj.com/politics/policy/government...,131


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
222,178,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...
103,84,2025-10-01-17-43-30 +0000,nypost,What will ‘TrumpRx’ actually mean for patients...,https://nypost.com/2025/10/01/health/what-will...
16,72,2025-10-01-20-31-07 +0000,wapo,Israel steps up Gaza City assault as Hamas wei...,https://www.washingtonpost.com/world/2025/10/0...
323,72,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
248,61,2025-10-01-10-00-00 +0000,nypost,Howard Lutnick tells ‘Pod Force One’ ex-neighb...,https://nypost.com/2025/10/01/us-news/howard-l...
15,60,2025-10-01-20-31-19 +0000,nypost,‘Wheel of Fortune’ winner kept her $1M prize s...,https://nypost.com/2025/10/01/lifestyle/wheel-...
149,46,2025-10-01-15-20-56 +0000,latimes,Supreme Court puts off decision on whether Tru...,https://www.latimes.com/politics/story/2025-10...
205,44,2025-10-01-12-00-00 +0000,wsj,Some Democrats have found an answer to Preside...,https://www.wsj.com/politics/policy/democrats-...
178,44,2025-10-01-14-05-00 +0000,wsj,White House nominees to lead the BLS and the C...,https://www.wsj.com/politics/policy/white-hous...
172,39,2025-10-01-14-17-00 +0000,wsj,Jeffrey Epstein never got kicked out of Wall S...,https://www.wsj.com/finance/jeffrey-epstein-we...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
